In [71]:
import os
from dotenv import load_dotenv
load_dotenv()

from src.data_generators.GetDataFromMongo import GetDataFromMongo

In [3]:
LLM_APP_ID = "6585977b875bdeef2f737584"

In [72]:
mongo = GetDataFromMongo(os.getenv("MONGO_CONNECTION_STRING"))
data = mongo(llm_app_id=LLM_APP_ID)

Data will be fetched from ProdSemantics
Query returned 284 results...
Formatting chunks...


 79%|███████▊  | 223/284 [00:34<00:09,  6.46it/s]

chunk_id '65a68c88a6ed2cba174cceea' did not match any chunk in ProdSemantics.DocumentChunk


 81%|████████  | 229/284 [00:35<00:08,  6.40it/s]

chunk_id '65a68c88a6ed2cba174cceea' did not match any chunk in ProdSemantics.DocumentChunk


 85%|████████▍ | 241/284 [00:37<00:06,  6.31it/s]

chunk_id '65a68c88a6ed2cba174cceea' did not match any chunk in ProdSemantics.DocumentChunk
chunk_id '65a68c88a6ed2cba174cceea' did not match any chunk in ProdSemantics.DocumentChunk


100%|██████████| 284/284 [00:44<00:00,  6.44it/s]


In [77]:
data[["query", "chunks", "answer"]].sample(20)

,query,chunks,answer
18,quele est l'adresse du cms de valence ?,[{'text': 'Les CMS (Centre Médicaux Sociaux) e...,"Je suis désolé, mais je ne peux pas vous fourn..."
220,comment accéder à drom einfos ?,[{'text': 'InfoRoute : Cette page permet d’...,"Pour accéder à Drôme Infos, le nouveau portail..."
47,Bonjour,"[{'text': 'Les cantons ', '_id': '6585694f875...",Bonjour ! Comment puis-je vous aider aujourd'h...
34,bonjour,"[{'text': 'Vivre à domicile ', '_id': '658569...",Bonjour ! Comment puis-je vous aider aujourd'h...
147,Comment on change d'établissement,"[{'text': 'Vivre en établissement ', '_id': '...","Pour changer d'établissement, il est nécessair..."
193,comment inscrire ma fille au transport scolaire ?,[{'text': 'Scolarisation #### En milieu ordin...,Pour inscrire votre fille au transport scolair...
152,quelle est la voiture la plus cher du monde ?,"[{'text': 'Le nucléaire ', '_id': '6585694e87...","Je n'ai pas encore la réponse, mais j'apprends..."
124,j'ai un probléme électrique sur ma route,[{'text': 'Problèmes routiers : Cette page ...,Si vous rencontrez un problème électrique sur ...
27,registres numérisés,[{'text': 'Les archives départementales 32 ki...,Les Archives départementales de la Drôme propo...
204,quels sont vos horaires d'ouverture,[{'text': 'Musée départemental de la Résistanc...,Je n'ai pas encore la réponse mais j'apprends ...


In [76]:
data["query"].iloc[78]

'Besoin de déposer un recherche de mon mari a Montélimar'